# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI

In [15]:
# setup environment
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
model = os.getenv("GEMINI_FM")
base_url = os.getenv("GEMINI_BASE_URL")

if(api_key and model and base_url) is None:
    raise ValueError("One or more environment variables are not set. Please check GEMINI_API_KEY, GEMINI_MODEL, and GEMINI_BASE_URL.")
else:
    print("Environment variables loaded successfully.")


Environment variables loaded successfully.


In [16]:
# PROMPT
code_assistand_system_prompt = """
    You are a helpful coding assistant, expertise in java, python and other scripting and programming languages.
    You help users by analysing code, sharing programming laguage name and then explaining it, highlight bugs and suggesting improvements.
"""

question = """
    Please explain what this code does and why:
    yield from {book.get("author") for book in books if book.get("author")}
"""

In [17]:
# Common code
payload = [
    {"role": "system", "content": code_assistand_system_prompt},
    {"role": "user", "content": question}
]

gemini = OpenAI(api_key=api_key, base_url=base_url)

In [18]:
response = gemini.chat.completions.create(model=model,messages=payload)

print(response.choices[0].message.content)

This code is written in **Python**.

Let's break down what it does and why.

## What this code does:

This single line of code, typically found within a **generator function**, performs the following actions:

1.  **Iterates through a collection of books:** It assumes there's an iterable variable named `books` (e.g., a list of dictionaries), where each dictionary represents a book.
2.  **Safely extracts author names:** For each `book` dictionary, it tries to get the value associated with the key `"author"` using `book.get("author")`. This method is safe because if the `"author"` key doesn't exist in a `book`, it returns `None` instead of raising a `KeyError`.
3.  **Filters out invalid author entries:** The `if book.get("author")` condition checks if the retrieved author value is "truthy." This means it will include authors that are non-empty strings (e.g., "J.K. Rowling") but will exclude `None` (if the key was missing) or an empty string `""` (if the key existed but had no value).
4. 

## Streaming response

In [20]:
# new import
from IPython.display import Markdown, display, update_display

In [22]:
stream_resp = gemini.chat.completions.create(model=model, messages=payload, stream=True)

display_handle = display(Markdown(""), display_id=True)

response = ""
for chunk in stream_resp:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

This Python code snippet is a concise and powerful way to extract unique author names from a collection of "book" dictionaries, and then yield them one by one.

Let's break down what it does and why:

**Programming Language:** Python

---

### What the Code Does

This single line of code performs the following actions:

1.  **Iterates through `books`:** It assumes `books` is an iterable (like a list) containing dictionary-like objects, where each object represents a book.
2.  **Extracts Author Names (Safely):** For each `book`, it tries to get the value associated with the key `"author"` using `book.get("author")`. The `.get()` method is safe because it returns `None` if the `"author"` key doesn't exist, instead of raising a `KeyError`.
3.  **Filters out Invalid Authors:** The `if book.get("author")` condition checks if the retrieved author name is "truthy." In Python, `None`, empty strings (`""`), and other empty collections (like `[]` or `{}`) are considered "falsy." So, this condition ensures that only books with an actual, non-empty author name are considered.
4.  **Creates a Set of Unique Authors:** The `{ ... for ... if ... }` syntax is a **set comprehension**. It creates a set where each element is the author name extracted in the previous steps. A key characteristic of a set is that it only stores **unique** elements; any duplicate author names will automatically be discarded.
5.  **Yields Authors One by One:** The `yield from` statement is used in a generator function. It takes an iterable (in this case, the set of unique author names) and effectively yields each item from that iterable individually. This means the authors are produced one at a time as they are requested, rather than building a complete list in memory first (though the set itself *is* built first).

**In summary:** It generates unique, non-empty author names found in a collection of book data.

---

### Why it's Used

This specific combination of Python features is used for several reasons:

1.  **Uniqueness:** The primary reason to use a **set comprehension** (`{...}`) here is to efficiently get only the unique author names. If a list comprehension were used instead (`[...]`), you would get duplicate author names if the same author appeared in multiple books.
2.  **Robustness/Error Handling:** Using `book.get("author")` instead of `book["author"]` makes the code more robust. If some `book` dictionaries in the `books` collection happen to be missing the `"author"` key, `book.get("author")` will return `None`, preventing a `KeyError` that would crash the program.
3.  **Filtering "Falsy" Values:** The `if book.get("author")` condition ensures that only meaningful author names are included. Authors that are `None` (missing) or empty strings (`""`) are ignored, providing cleaner data.
4.  **Conciseness and Readability (Pythonic):** Set comprehensions are a very "Pythonic" way to create sets based on existing iterables. They are often more compact and readable than a traditional `for` loop with `set.add()` calls.
5.  **Generator Delegation (`yield from`):**
    *   `yield from` is a feature of generator functions (a function containing `yield`).
    *   It's a convenient way to delegate iteration to another iterable. Instead of looping through the set and `yield`ing each item manually, `yield from` does it for you.
    *   While the set comprehension itself creates the entire set in memory upfront, `yield from` then iterates over that set and produces its elements one at a time, which can be useful if the consumer of these authors also wants to process them lazily.

---

### Example

Let's imagine you have a list of books like this:

```python
books = [
    {"title": "Book A", "author": "Alice"},
    {"title": "Book B", "author": "Bob"},
    {"title": "Book C", "author": "Alice"},
    {"title": "Book D"}, # Missing author
    {"title": "Book E", "author": ""}, # Empty author
    {"title": "Book F", "author": "Charlie"},
]

def get_unique_authors(book_list):
    yield from {book.get("author") for book in book_list if book.get("author")}

# To use it:
authors_generator = get_unique_authors(books)
for author in authors_generator:
    print(author)
```

**Output:**

```
Alice
Bob
Charlie
```

Notice:
*   "Alice" appears only once.
*   The books with missing (`"Book D"`) or empty (`"Book E"`) authors are ignored.
*   The order might not be the same every time, as sets are inherently unordered.